# Isle of Wight Local Elections

Simple scribbles around Isle of Wight local election candidate data.

Reusing code and ideas from:

- [Questioning Election Data to See if It Has a Story to Tell](https://blog.ouseful.info/2013/05/05/questioning-election-data-to-see-if-it-has-a-story-to-tell/)
- [More Storyhunting Around Local Elections Data Using Gephi – To What Extent Do Candidates Support Each Other?](https://blog.ouseful.info/2013/05/08/more-storyhunting-around-local-elections-data-using-gephi-to-what-extent-do-candidates-support-each-other/)


Note that this is a bit ropey and requires you to work through the steps.

You may also need to install some additional packages along the way...

### Configuring the scraper source and related filenames

In [1]:
#Define scraper sqlite database filename
dbname= "iwdb.sqlite"

In [2]:
#Optionally delete the previous instance
!rm {dbname}

In [3]:
#Filename for map of candidates
mapname='iwmap.html'

In [3]:
# The localarea is a search keyword we look for in the address of companies
# that may be associated with directors with the same exact name as a candidate
# Use it to limit display of companies to companies with addresses that contain that keyword
localarea='Wight'

In [4]:
#Candidates filename
candsfilename='wightcands.csv'

In [5]:
#Supporters filename
supportersfilename='wightsupporters.csv'

In [7]:
#filename for companies
companiesfilename='wightcos.csv'

In [4]:
#Path to the Notice of election file (note, this can be the path/name of a local file)
#url='https://www.south-norfolk.gov.uk/sites/default/files/Notice%20of%20Poll.pdf'
year=2017
#url='https://dl.dropboxusercontent.com/u/1156404/1174-Notice%20of%20Poll%20-%20IOWC%20May%202013.pdf'
url='https://www.iwight.com/azservices/documents/1174-Notice-of-Poll-IOWC-2017.pdf'

In [58]:
#Companies House API token
#Available from: https://developer.companieshouse.gov.uk/api/docs/index/gettingStarted/apikey_authorisation.html
CH_API_TOKEN='YOUR KEY'

## Install Necessary Packages

In [10]:
#Set the path to the pip installer for your Python kernel

#It may be available on your path directly, or you may need to specify a path, as below, or pip3
#pip='~/anaconda/bin/pip'
#pip='pip3'

pip='pip'

In [11]:
#Install required packages
!{pip} install lxml
!{pip} install scraperwiki

!{pip} install pandas
!{pip} install networkx

!{pip} install geocoder
!{pip} install folium
!{pip} install uk-postcode-utils

## Scraping the Data

In [2]:
#Configure name of scraper sqlite database file
import os
os.environ["SCRAPERWIKI_DATABASE_NAME"] ='sqlite:///{}'.format(dbname)

In [29]:
def pdftoxml(pdfdata, options=""):
    """converts pdf file to xml file"""
    pdffout = tempfile.NamedTemporaryFile(suffix='.pdf')
    pdffout.write(pdfdata)
    pdffout.flush()

    xmlin = tempfile.NamedTemporaryFile(mode='r', suffix='.xml')
    tmpxml = xmlin.name  # "temph.xml"

    cmd = 'pdftohtml -xml -nodrm -zoom 1.5 -enc UTF-8 -noframes %s "%s" "%s"' % (
        options, pdffout.name, os.path.splitext(tmpxml)[0])
    # can't turn off output, so throw away even stderr yeuch
    cmd = cmd + " >/dev/null 2>&1"
    os.system(cmd)

    pdffout.close()
    #xmlfin = open(tmpxml)
    
    root = lxml.etree.parse(tmpxml)#lxml.etree.fromstring(xmldata)
    xmlin.close()
    
    pages = list(root.getroot())
    return pages

In [30]:
# SCRAPER CODE REUSED FROM A LONG TIME AGO, WITH MINOR TWEAKS

#code from https://classic.scraperwiki.com/scrapers/iw_poll_notices_scrape/
#with a couple of minor tweaks - seach for strings using 'in' rather than 'startwsith'
import scraperwiki
import requests, lxml.etree
import tempfile

#Current local election notice of election PDF for Isle of Wight - looks to be same format-ish as before
#url='https://www.iwight.com/azservices/documents/1174-Notice-of-Poll-IOWC-2017.pdf'


#Read in the Notice of Poll PDF
pdfdata = requests.get(url).content

#Convert PDF to XML - this breaks in Python 3?
pages = pdftoxml(pdfdata)#scraperwiki.pdftoxml(pdfdata)


In [31]:
# this function has to work recursively because we might have "<b>Part1 <i>part 2</i></b>"
def gettext_with_bi_tags(el):
    res = [ ]
    if el.text:
        res.append(el.text)
    for lel in el:
        res.append("<%s>" % lel.tag)
        res.append(gettext_with_bi_tags(lel))
        res.append("</%s>" % lel.tag)
        if el.tail:
            res.append(el.tail)
    return "".join(res).strip()


#Scrape the separate pages
#print(pages)
for page in pages:
    data={'stations':[]}
    phase=0
    for el in page:
        #print(el.attrib, gettext_with_bi_tags(el))
        if 'Election of' in gettext_with_bi_tags(el):
            phase=1
            continue
        tmp=gettext_with_bi_tags(el).strip()
        if phase==1:
            if tmp=='':pass
            else:
                data['ward']=tmp
                phase=phase+1
        elif phase==2:
            if 'Proposers' in tmp:
                phase=3
                record={'candidate':[],'address':[],'desc':[],'proposers':[],'seconders':[]}
                data['list']=[]
                continue
        elif phase==3:
            if tmp.strip()=='':
                phase=4
                #print('-------------------------------')
                data['list'].append(record)
                continue
            elif int(el.attrib['left'])<100:
                if record['address']!=[]:
                    data['list'].append(record)
                    record={'candidate':[],'address':[],'desc':[],'proposers':[],'seconders':[]}
                record['candidate'].append(tmp)
            elif int(el.attrib['left'])<300: record['address'].append(tmp)
            elif int(el.attrib['left'])<450: record['desc'].append(tmp)
            elif int(el.attrib['left'])<600:
                if tmp.startswith('('): record['proposers'][-1]=record['proposers'][-1]+' '+tmp
                elif len(record['proposers'])>0 and record['proposers'][-1].strip().endswith('-'): record['proposers'][-1]=record['proposers'][-1]+tmp
                elif len(record['proposers'])>0 and record['proposers'][-1].strip().endswith('.'): record['proposers'][-1]=record['proposers'][-1]+' '+tmp
                else: record['proposers'].append(tmp)
            elif int(el.attrib['left'])<750:
                if tmp.startswith('('): record['seconders'][-1]=record['seconders'][-1]+' '+tmp
                elif len(record['seconders'])>0 and record['seconders'][-1].strip().endswith('-'): record['seconders'][-1]=record['seconders'][-1]+tmp
                elif len(record['seconders'])>0 and record['seconders'][-1].strip().endswith('.'): record['seconders'][-1]=record['seconders'][-1]+' '+tmp
                else: record['seconders'].append(tmp)
        elif phase==4:
            if 'persons entitled to vote' in tmp:
                phase=5
                record={'station':[],'range':[]}
                continue
        elif phase==5: #Not implemented... TO DO
            #print(el.attrib, gettext_with_bi_tags(el))
            if tmp.strip()=='':
                data['stations'].append(record)
                break #The following bits are broken...
            #need to add situation
            elif int(el.attrib['left'])<100:
                if record['range']!=[]:
                    data['stations'].append(record)
                    record={'situation':[],'station':[],'range':[]}
                record['station'].append(tmp)
            elif int(el.attrib['left'])>300:
                record['range'].append(tmp)
    #print(data)
    tmpdata=[]
    for station in data['stations']:
        tmpdata.append({'ward':data['ward'],
                        #'situation':' '.join(station['situation']),
                        'station':' '.join(station['station']),
                        'range':' '.join(station['range'])})
    scraperwiki.sqlite.save(unique_keys=[], table_name='stations', data=tmpdata)
    tmpdata=[]
    tmpdata2=[]
#'desc': ['The Conservative Party', 'Candidate'], 'candidate': ['OULTON', 'Erica'], 'address': ['Blandings, Horringford,', 'Arreton, IW, PO30 3AP']
    for candidate in data['list']:
        tmpdata.append( {'year':year,
                         'ward':data['ward'],
                         'candidate':' '.join(candidate['candidate']),#.encode('ascii','ignore'),
                         'address':' '.join(candidate['address']),
                         'desc':' '.join(candidate['desc']) } )
        party=' '.join(candidate['desc']).replace('Candidate','').strip()
        cand=' '.join(candidate['candidate'])#.encode('ascii','ignore')
        cs=cand.strip(' ').split(' ')
        if len(cs)>2:
            cand2=cs[:2]
            for ci in cs[2:]:
                cand2.append(ci[0]+'.')
        else: cand2=cs
        ctmp=cand2[0]
        cand2.remove(ctmp)
        cand2.append(ctmp.title())
        candi=' '.join(cand2)#.encode('ascii','ignore')
        for proposer in candidate['proposers']:
            if proposer.find('(+)')>-1:
                proposer=proposer.replace('(+)','').strip()
                typ='proposer'
            else:typ='assentor'
            tmpdata2.append({ 'year':year,
                             'ward':data['ward'],
                             'candidate':cand, 
                             'candinit':candi,
                             'support':proposer,
                             'role':'proposal',
                             'typ':typ,
                             'desc':party}.copy())
        for seconder in candidate['seconders']:
            if seconder.find('(++)')>-1:
                seconder=seconder.replace('(++)','').strip()#.encode('ascii','ignore')
                typ='seconder'
            else:typ='assentor'
            tmpdata2.append({ 'year':year,
                             'ward':data['ward'],
                             'candidate':cand, 
                             'candinit':candi,
                             'support':seconder,
                             'role':'seconding',
                             'typ':typ, 
                             'desc':party }.copy())

    scraperwiki.sqlite.save(unique_keys=[], table_name='candidates', data=tmpdata)
    scraperwiki.sqlite.save(unique_keys=[], table_name='support', data=tmpdata2)

In [32]:
#Check database tables
scraperwiki.sql.show_tables()

{'candidates': 'CREATE TABLE candidates (\n\tyear BIGINT, \n\tward TEXT, \n\tcandidate BLOB, \n\taddress TEXT, \n\t"desc" TEXT\n)',
 'stations': 'CREATE TABLE stations (\n\tward TEXT, \n\tstation TEXT, \n\trange TEXT\n)',
 'support': 'CREATE TABLE support (\n\tyear BIGINT, \n\tward TEXT, \n\tcandidate TEXT, \n\tcandinit BLOB, \n\tsupport TEXT, \n\trole TEXT, \n\ttyp TEXT, \n\t"desc" TEXT\n)'}

## Querying the Data

In [27]:
%matplotlib inline
import pandas as pd
import sqlite3

#Create a connection to the database so we can query it using pandas
conn = sqlite3.connect(dbname)

In [28]:
#TO DO... scraper not complete for extracting poll station info
stations= pd.read_sql_query("SELECT * FROM stations", conn)
stations.head(5)

,ward,station,range
0,Arreton and Newchurch,,
1,Arreton and Newchurch,,
2,Binstead and Fishbourne,,
3,"Brading, St Helens and Bembridge",,
4,Carisbrooke,,


In [16]:
candidates = pd.read_sql_query("SELECT * FROM candidates WHERE year={year}".format(year=year), conn)

#Clean the data a bit - should maybe do this as part of the scrape, or provide a "clean col" as part of scrape
candidates['desc']=candidates['desc'].str.replace('The ','').str.replace(' Candidate','')
candidates.head(5)

,desc,ward,address,candidate,year
0,Conservative Party,Arreton and Newchurch,"Blandings, Horringford, Arreton, IW, PO30 3AP",OULTON Erica,2013
1,Independent,Arreton and Newchurch,"Popes Farm, Newchurch, Sandown, IW, PO36 0NF",RICHARDS Colin,2013
2,UKIP,Arreton and Newchurch,"Buckbury Farm, Buckbury Lane, Newport, PO30 2NL",SHERFIELD Ian,2013
3,Labour Party,Binstead and Fishbourne,"Flat 7, The Cedars, 50 West Hill Road, Ryde, I...",CURTIS Katie,2013
4,UKIP,Binstead and Fishbourne,"28 High Street, Wootton Bridge, Ryde, PO33 4PL",DARNELL Alan,2013


In [18]:
candidates['candidate'].unique()

array([u'BELFITT Sandy', u'HAYDEN Ali', u'MACLENNAN Iain Ross',
       u'MOSDELL Clare Louise', u'POINTING Joshua Stuart Frederick',
       u'COOK Jacqueline Eleanor', u'CORY Christopher', u'GLAZEBROOK John',
       u'KILPATRICK John Clive', u'PITCHER Linda Pauline',
       u'WARLOW Ivor George', u'BACON Jonathan Francis',
       u'BARTRUM JNR Roger', u'GRANEY John Henry', u'KENDALL Gordon',
       u'MURWILL Michael Anthony', u'STREET Hugh Thomas',
       u'TYNDALL Brian George', u'HOBART John Vere', u'MAY Imogen Thea',
       u'VERSTRAETEN Liam Norman Peter', u'LANGAN Andrew Michael',
       u'MCKEAN Jane Alison', u'SEELY Bob', u'CRITCHISON Claire Leah',
       u'LOADER Wendy Christine', u'STEWART David Kevin', u'HALDENBY Ben',
       u'OLIVER Neil', u'PEACEY WILCOX Lora', u'WALTERS David Kenneth',
       u'ADAMS Anni', u'BARTRUM Jean Margaret', u'BERTIE Paul Michael',
       u'LYNDEN-BELL Rose', u'BOSSMAN Will',
       u'NICHOLSON John Ulysses Anthony', u'TENNANT Laurie',
       u'FU

In [19]:
#Save supportes data file
candidates.to_csv(candsfilename,index=False)

## Geocode and Map the Candidates' Addresses

In [20]:
import geocoder
def gc(address):
    g=geocoder.google(address)
    try:
        return '{},{}'.format(g.latlng[0],g.latlng[1])
    except:
        pass
    try:
        pc=address.split(',')[-1].strip()
        if pc.startswith('PO'):
            g=geocoder.google(pc)
            return '{},{}'.format(g.latlng[0],g.latlng[1])
        else: return ''
    except:
        return ''

candidates['latlong']=candidates['address'].apply(gc)

In [21]:
candidates.head()

,ward,desc,candidate,address,latlong
0,Arreton and Newchurch,Liberal Democrat,BELFITT Sandy,"2 The Limes, Priory Road, Shanklin, Isle of Wi...","50.6223964,-1.1783662"
1,Arreton and Newchurch,Independent,HAYDEN Ali,"71 St Michaels Avenue, Ryde, I.O.W, PO33 3EH","50.7180033,-1.1687424"
2,Arreton and Newchurch,Labour Party,MACLENNAN Iain Ross,"Station House, Main Road, Alverstone, Sandown,...","50.6679259,-1.183218"
3,Arreton and Newchurch,Conservative Party,MOSDELL Clare Louise,"1 Wentworth Close, Shanklin, Isle of Wight, PO...","50.6264453,-1.1889273"
4,Arreton and Newchurch,Green Party,POINTING Joshua Stuart Frederick,"27F Holyrood Street, Newport, Isle of Wight, P...","50.7008209,-1.2939878"


In [22]:
import folium

def add_marker(row,fmap):
    if row['latlong']!='':
        lat=row['latlong'].split(',')[0]
        long=row['latlong'].split(',')[1]
        folium.Marker([lat, long], popup='{}, ({})\n{}'.format(row['candidate'],row['desc'],row['ward'])).add_to(fmap)
    
#Create a map centered on the postcode location at a particular zoom level
#Really crude centrepoint for map
centrelatlong=candidates[candidates['latlong']!=''].iloc[0]['latlong'].split(',')


localmap = folium.Map(location=centrelatlong, zoom_start=11)

candidates.apply(lambda x: add_marker(x, localmap), axis=1)
localmap

In [24]:
#save the map
localmap.save(mapname)

## Guess at Whether Candidates Live In-Ward or Out-of-Ward

This could be a bit ropey - really need to check we are using the correct administrative geographies.

Use a service like [MapIt](https://mapit.mysociety.org/) or [postcodes.io](https://api.postcodes.io/) to find ward from postcode, then compare this to the name of the ward they are standing in.

In [26]:
import requests
from ukpostcodeutils import validation

def getpc(addr):
    ''' Crude attempt to find postcode in address; conventionally last part of address in this dataset? '''
    pc=addr.split(',')[-1].strip()
    if validation.is_valid_postcode(pc.replace(' ','')): return pc
    return ''

def getpcward(pc):
    ''' Lookup a ward from a postcode using postcodes.io '''
    if pc!='':
        try:
            return requests.get('https://api.postcodes.io/postcodes/{}'.format(pc.replace(' ',''))).json()['result']['admin_ward']
        except:
            return ''
    return ''

#Extract the postcode - conventionally, it looks like postcode is last part of address so guess at that
candidates['postcode']=candidates['address'].apply(getpc)

#Get the list of unique postcodes for candidates and lookup the corresponding ward
pcwards={pc: getpcward(pc) for pc in candidates['postcode'].unique()}

candidates['pcward']=candidates['postcode'].map(pcwards)
candidates.head()
#If this doesn't catch anything - could also try to use latlong where no postcode available...

,ward,desc,candidate,address,latlong,postcode,pcward
0,Arreton and Newchurch,Liberal Democrat,BELFITT Sandy,"2 The Limes, Priory Road, Shanklin, Isle of Wi...","50.6223964,-1.1783662",PO37 6RE,Shanklin South
1,Arreton and Newchurch,Independent,HAYDEN Ali,"71 St Michaels Avenue, Ryde, I.O.W, PO33 3EH","50.7180033,-1.1687424",PO33 3EH,"Havenstreet, Ashey and Haylands"
2,Arreton and Newchurch,Labour Party,MACLENNAN Iain Ross,"Station House, Main Road, Alverstone, Sandown,...","50.6679259,-1.183218",PO36 0EZ,Arreton and Newchurch
3,Arreton and Newchurch,Conservative Party,MOSDELL Clare Louise,"1 Wentworth Close, Shanklin, Isle of Wight, PO...","50.6264453,-1.1889273",PO37 6QD,Shanklin South
4,Arreton and Newchurch,Green Party,POINTING Joshua Stuart Frederick,"27F Holyrood Street, Newport, Isle of Wight, P...","50.7008209,-1.2939878",PO30 5AZ,Newport Central


In [27]:
#display in ward - so Ward they're standing in is same as ward of their address
candidates[candidates['ward']==candidates['pcward']].head()

,ward,desc,candidate,address,latlong,postcode,pcward
2,Arreton and Newchurch,Labour Party,MACLENNAN Iain Ross,"Station House, Main Road, Alverstone, Sandown,...","50.6679259,-1.183218",PO36 0EZ,Arreton and Newchurch
5,Binstead and Fishbourne,Green Party,COOK Jacqueline Eleanor,"37 Binstead Lodge Road, Binstead, PO33 3TN","50.7216832,-1.1879094",PO33 3TN,Binstead and Fishbourne
7,Binstead and Fishbourne,Labour Party,GLAZEBROOK John,"11 The Mall, Binstead, PO33 3SF","50.7248011,-1.1905137",PO33 3SF,Binstead and Fishbourne
8,Binstead and Fishbourne,Conservative Party,KILPATRICK John Clive,"9 The Old Forge Close, Binstead, PO33 3PL","50.7283605,-1.1861351",PO33 3PL,Binstead and Fishbourne
10,Binstead and Fishbourne,Independent,WARLOW Ivor George,"21 Chapel Road, Binstead, Isle of Wight, PO33 3RT","50.7262264,-1.1882869",PO33 3RT,Binstead and Fishbourne


In [28]:
#display out ward - so Ward they're standing in is not the same as ward of their address
candidates[candidates['ward']!=candidates['pcward']]#.head()

,ward,desc,candidate,address,latlong,postcode,pcward
0,Arreton and Newchurch,Liberal Democrat,BELFITT Sandy,"2 The Limes, Priory Road, Shanklin, Isle of Wi...","50.6223964,-1.1783662",PO37 6RE,Shanklin South
1,Arreton and Newchurch,Independent,HAYDEN Ali,"71 St Michaels Avenue, Ryde, I.O.W, PO33 3EH","50.7180033,-1.1687424",PO33 3EH,"Havenstreet, Ashey and Haylands"
3,Arreton and Newchurch,Conservative Party,MOSDELL Clare Louise,"1 Wentworth Close, Shanklin, Isle of Wight, PO...","50.6264453,-1.1889273",PO37 6QD,Shanklin South
4,Arreton and Newchurch,Green Party,POINTING Joshua Stuart Frederick,"27F Holyrood Street, Newport, Isle of Wight, P...","50.7008209,-1.2939878",PO30 5AZ,Newport Central
6,Binstead and Fishbourne,Liberal Democrat,CORY Christopher,"32 Palmers Road, Wootton Bridge, Isle of Wight...","50.7272022,-1.2400093",PO33 4NA,Wootton Bridge
9,Binstead and Fishbourne,UKIP,PITCHER Linda Pauline,"19 Norman Way, Wootton, Isle of Wight, PO33 4NJ","50.7301826,-1.2358907",PO33 4NJ,Wootton Bridge
11,"Brading, St Helens and Bembridge",Independent,BACON Jonathan Francis,"Bagwich House, Bagwich Lane, Godshill, Isle of...","50.632706,-1.2694016",PO38 3JY,Godshill and Wroxall
12,"Brading, St Helens and Bembridge",Green Party,BARTRUM JNR Roger,"7 Granville Road, Cowes, PO31 7JD","50.761516,-1.3010416",PO31 7JD,Cowes North
15,"Brading, St Helens and Bembridge",Conservative Party,MURWILL Michael Anthony,"Ananda House, 122 Green Lane, Lake, IOW, PO36 9NL","50.642637,-1.1716492",PO36 9NL,Lake South
18,Carisbrooke,Conservative Party,HOBART John Vere,"South Farm Cottage, Sandy Lane, Newport, IOW, ...","50.6736436,-1.2989951",PO30 3EA,Newport South


In [29]:
candidates.to_csv('geo'+candsfilename,index=False)

## Chart cross-support

From the table of supporters, we can try to identify candidates who support other candidates.

In [17]:
#Preview the supporters table
supporters = pd.read_sql_query("SELECT * FROM support WHERE year={year}".format(year=year), conn)
#Clean the data a bit
supporters['desc']=supporters['desc'].str.replace('The ','').str.replace(' Candidate','')
supporters.head(5)

supporters.head(5)

,candinit,candidate,support,role,year,ward,typ,desc
0,Sandy Belfitt,BELFITT Sandy,Tamara Purves,proposal,2017,Arreton and Newchurch,proposer,Liberal Democrat
1,Sandy Belfitt,BELFITT Sandy,Melanie F Amey,proposal,2017,Arreton and Newchurch,assentor,Liberal Democrat
2,Sandy Belfitt,BELFITT Sandy,Philip F. Warner,proposal,2017,Arreton and Newchurch,assentor,Liberal Democrat
3,Sandy Belfitt,BELFITT Sandy,Richard A Hill,proposal,2017,Arreton and Newchurch,assentor,Liberal Democrat
4,Sandy Belfitt,BELFITT Sandy,Kate I. Gilliam,proposal,2017,Arreton and Newchurch,assentor,Liberal Democrat


In [31]:
#Save supportes data file
supporters.to_csv(supportersfilename,index=False)

In [29]:
#Find the unique parties
supporters['desc'].unique()

array([u'Liberal Democrat', u'Labour Party', u'Conservative Party',
       u'Green Party'], dtype=object)

In [30]:
colourmap={'Liberal Democrat':'yellow', 'Independent':'black', 'Labour Party':"red",
       'Conservative Party':'blue', 'Green Party':'green', 'UKIP':'purple',
       'Labour and Co- operative Party':'red'}

In [31]:
#Create a graph
import networkx as nx

In [32]:
#G=nx.from_pandas_dataframe(supporters, 'support', 'candinit')
def build_graph(row,DG):
    DG.add_node(row['support'],color=colourmap[row['desc']])
    DG.add_node(row['candinit'],color=colourmap[row['desc']])
    DG.add_edge(row['support'],row['candinit'],color=colourmap[row['desc']])
    return

DG=nx.DiGraph()
supporters.apply(lambda x: build_graph(x,DG), axis=1);

In [33]:
nodes = DG.nodes()
edges = DG.edges()

In [34]:
#filter on people who are supported and who support
supports_deg = DG.out_degree(nodes)
supported_deg = DG.in_degree(nodes)
supports = [n for n in supports_deg if supports_deg[n]]
supported = [n for n in supported_deg if supported_deg[n]]

GG=nx.DiGraph()
#Merge the egographs of people of people who support and are supported
for s2 in list(set(supports).intersection(set(supported))):
    GG=nx.compose(GG,nx.ego_graph(DG,s2,5))

In [35]:
#http://bl.ocks.org/brinrosenthal/raw/cfb0e12f113d55551a45d530527baedf/
import visJS2jupyter.visJS_module
nodes = GG.nodes()
edges = GG.edges()

pos = nx.spring_layout(GG)
nodes_dict = [{"id":n,"color":GG.node[n]['color'],
              "x":pos[n][0],
              "y":pos[n][1]} for n in nodes]
node_map = dict(zip(nodes,range(len(nodes))))  # map to indices for source/target in edges

edges_dict = [{"source":node_map[edges[i][0]], "target":node_map[edges[i][1]], "color":GG[edges[i][0]][edges[i][1]]['color'],
              "title":'test'} for i in range(len(edges))]

visJS2jupyter.visJS_module.visjs_network(nodes_dict,edges_dict)

In [53]:
#Support between connected candidates where a candidate supports another candidate
for e in edges:
    print(e)

(u'Elana Katz', u'Pam Reekie')


## Look for people with same name offering support to multiple candidates

Does it look like the same person is supporting more than one candidate?

In [32]:
supporters[supporters['support'].isin(supporters[supporters.duplicated(subset='support')]['support'].unique())]

,candinit,role,candidate,support,typ,ward,desc
150,Michael A. Murwill,proposal,MURWILL Michael Anthony,Christopher R Bristow,proposer,"Brading, St Helens and Bembridge",Conservative Party
171,Brian G. Tyndall,proposal,TYNDALL Brian George,Christopher R Bristow,assentor,"Brading, St Helens and Bembridge",Conservative Party


In [33]:
#Another way of doing that
pd.read_sql_query("SELECT * FROM support WHERE support=(SELECT support FROM support GROUP BY support HAVING COUNT(*)>1)", conn)


,candinit,role,candidate,support,typ,ward,desc
0,Michael A. Murwill,proposal,MURWILL Michael Anthony,Christopher R Bristow,proposer,"Brading, St Helens and Bembridge",Conservative Party
1,Brian G. Tyndall,proposal,TYNDALL Brian George,Christopher R Bristow,assentor,"Brading, St Helens and Bembridge",Conservative Party


## Look for candidates sharing the same address

Does it look like multiple candidates share the same address?

In [34]:
## Multiple candidates from one address

pd.read_sql_query("SELECT * FROM candidates WHERE address=(SELECT address FROM candidates GROUP BY address HAVING COUNT(*)>1)", conn)


,ward,desc,candidate,address
0,Binstead and Fishbourne,UKIP,PITCHER Linda Pauline,"19 Norman Way, Wootton, Isle of Wight, PO33 4NJ"
1,Wootton Bridge,UKIP,PITCHER Daryll,"19 Norman Way, Wootton, Isle of Wight, PO33 4NJ"


## Look for the same party standing multiple candidates in the same ward

Does it look like the same party is supporting more than one candidate in a particular ward?

In [35]:
## Multiple Candidates for a Party in Same Ward

pd.read_sql_query("SELECT ward,desc, count(*) FROM support WHERE typ='proposer' GROUP BY ward,desc HAVING COUNT(*)>1", conn)


,ward,desc,count(*)
0,"Brading, St Helens and Bembridge",Conservative Party,2
1,"Brading, St Helens and Bembridge",Independent,2
2,East Cowes,Independent,2
3,Newport West,Independent,2
4,Totland,Independent,2


## Companies House / OpenCorporates Lookup

Check to see whether the names of candidates are also possible company directors.

Could also do a check to see if they are charity trustees, bankrupt, disqualified director, registered licensee on any IW Council registers etc etc.

__NOTE THAT THE FOLLOWING DOES NOT GUARANTEE OR NECESSARILY IMPLY THAT THE PERSON NAMED AS STANDING IS THE SAME PERSON AS A SIMILARLY NAMED COMPANY OFFICER.__

In [36]:
import urllib2, base64, json
from urllib import urlencode
from time import sleep

def url_nice_req(url,t=300):
    try:
        return urllib2.urlopen(url)
    except urllib2.HTTPError, e:
        if e.code == 429:
            print("Overloaded API, resting for a bit...")
            sleep(t)
            return url_req(url)
        
#Inspired by http://stackoverflow.com/a/2955687/454773
def ch_request(CH_API_TOKEN,url,args=None):
    if args is not None:
        url='{}?{}'.format(url,urlencode(args))
    request = urllib2.Request(url)
    # You need the replace to handle encodestring adding a trailing newline 
    # (https://docs.python.org/2/library/base64.html#base64.encodestring)
    base64string = base64.encodestring('%s:' % (CH_API_TOKEN)).replace('\n', '')
    request.add_header("Authorization", "Basic %s" % base64string)   
    result = url_nice_req(request)

    #This is too hacky - need to see why it fails if it does
    if result is None:
        print('Oops: {}, {}'.format(url,result))
        return None
        
    j=json.loads(result.read())        
        
    return j

In [37]:
def ch_getAppointments(slug,location=None,typ='all',role='all',n=500,start_index=''):
    if len(slug.split('/'))==1:
        slug='/officers/{}/appointments'.format(slug)
    url= 'https://api.companieshouse.gov.uk{}'.format(slug)
    properties={'items_per_page':n,'start_index':start_index}
    a=ch_request(CH_API_TOKEN,url,properties)

    if a is None: return None
    
    if location is not None:
        a['items']=[i for i in a['items'] if location.lower() in i['address']['locality'].lower()]
    if typ=='current':
        a['items']=[i for i in a['items'] if 'resigned_on' not in i]
        a['items']=[i for i in a['items'] if 'company_status' in i['appointed_to'] and i['appointed_to']['company_status'] == 'active']
        #should possibly check here that len(co['items'])==co['active_count'] ?
    elif typ=='previous':
        a['items']=[i for i in a['items'] if 'resigned_on' in i]
    elif typ=='dissolved':
        a['items']=[i for i in a['items'] if 'company_status' in i['appointed_to'] and i['appointed_to']['company_status'] == 'dissolved']

    if role!='all':
        a['items']=[i for i in a['items'] if role==i['officer_role']]
    return a

In [38]:
def ch_searchOfficers(name,n=50,start_index='',company='',companies=False,exact=None):
    url= 'https://api.companieshouse.gov.uk/search/officers'
    properties={'q':name,'items_per_page':n,'start_index':start_index} 
    o=ch_request(CH_API_TOKEN,url,properties)
    
    if o is None: return o
    
    if exact=='forename':
        #This isn't right eg double barrelled surnames
        s=name.lower().split(' ')
        o['items'] = [i for i in o['items'] if i['title'].lower().split(' ')[0]==s[0] and i['title'].lower().split(' ')[-1]==s[-1]]
    elif exact=='fullname':
        o['items'] = [i for i in o['items'] if i['title'].lower()==name.lower()]
    if company != '':
        for p in o['items']:
            p['items'] = [i for i in ch_getAppointments(p['links']['self'])['items'] if company.lower() in i['appointed_to']['company_name'].lower()]
        o['items'] = [i for i in o['items'] if len(i['items'])]
    if companies:
        for p in o['items']:
            p['items'] = [i for i in ch_getAppointments(p['links']['self'])['items']]
        o['items'] = [i for i in o['items'] if len(i['items'])]
    return o

In [39]:
appointments=pd.DataFrame()
for c in candidates['candidate'].tolist():
    name=c.split()
    cand=' '.join(name[1:]+name[0:1])
    results=ch_searchOfficers(cand,n=50,exact='fullname')
    for result in results['items']:
        appointments=pd.concat([appointments,pd.DataFrame.from_dict([result])])

In [40]:
appointments[appointments['address_snippet'].str.contains(localarea)]

,address,address_snippet,appointment_count,date_of_birth,description,description_identifiers,kind,links,matches,snippet,title
0,"{u'locality': u'Sandown', u'premises': u'14', ...","14 Carter Street, Sandown, Isle Of Wight, Unit...",2,"{u'month': 7, u'year': 1968}",Total number of appointments 2 - Born July 1968,"[appointment-count, born-on]",searchresults#officer,{u'self': u'/officers/fydVKv0OBjmcZZ1gBLI5WNEy...,"{u'snippet': [], u'title': [1, 5, 7, 12, 14, 20]}",,Clare Louise MOSDELL
0,"{u'locality': u'Shanklin', u'premises': u'1', ...","1 Wentworth Close, Shanklin, Isle Of Wight, En...",3,"{u'month': 7, u'year': 1968}",Total number of appointments 3 - Born July 1968,"[appointment-count, born-on]",searchresults#officer,{u'self': u'/officers/HJVt3TrYxt5c560CfH6puryj...,"{u'snippet': [], u'title': [1, 5, 7, 12, 14, 20]}",,Clare Louise MOSDELL
0,"{u'premises': u'32', u'country': u'PO33 4NA', ...","32 Palmers Road, Wootton Bridge, Isle Of Wight...",1,"{u'month': 6, u'year': 1954}",Total number of appointments 1 - Born June 1954,"[appointment-count, born-on]",searchresults#officer,{u'self': u'/officers/Z24iEa7Gjht_C7CTodRTpxX0...,"{u'snippet': [], u'title': [1, 11, 13, 16]}",,Christopher CORY
0,"{u'premises': u'71', u'country': u'PO30 1UJ', ...","71 Pyle Street, Newport, Isle Of Wight, Englan...",0,"{u'month': 8, u'year': 1947}",Total number of appointments 0 - Born August 1947,"[appointment-count, born-on]",searchresults#officer,{u'self': u'/officers/myohqrg4-WSrjHGqcSD_ftmy...,"{u'snippet': [], u'title': [1, 5, 7, 13, 15, 21]}",,Linda Pauline PITCHER
0,"{u'locality': u'Ryde', u'premises': u'The Cast...","The Castle, Duver Road, St Helens, Ryde, Isle ...",2,"{u'year': 1966, u'month': 11}",Total number of appointments 2 - Born November...,"[appointment-count, born-on]",searchresults#officer,{u'self': u'/officers/YhnLA3NtOrOoY45kNbt1Cqa-...,"{u'snippet': [], u'title': [1, 8, 10, 16, 18, ...",,Jonathan Francis BACON
0,"{u'locality': u'Newport', u'premises': u'81', ...","81 Carisbrooke High Street, Newport, Isle Of W...",5,"{u'year': 1967, u'month': 12}",Total number of appointments 5 - Born December...,"[appointment-count, born-on]",searchresults#officer,{u'self': u'/officers/G7FFpRX2hA5CRiTN1Wm5yNrg...,"{u'snippet': [], u'title': [1, 4, 6, 11, 13, 1...",,Liam Norman Peter VERSTRAETEN
0,"{u'premises': u'24', u'country': u'PO31 7JD', ...","24 Granville Road, Cowes, Isle Of Wight, Unite...",4,"{u'month': 9, u'year': 1943}",Total number of appointments 4 - Born Septembe...,"[appointment-count, born-on]",searchresults#officer,{u'self': u'/officers/tMoXBCcTgzuickQ5NODB3Bd7...,"{u'snippet': [], u'title': [1, 5, 7, 13, 15, 21]}",,David Kenneth WALTERS
0,"{u'premises': u'39', u'country': u'PO31 8HP', ...","39 Crossfield Avenue, Cowes, Isle Of Wight, En...",1,"{u'year': 1949, u'month': 12}",Total number of appointments 1 - Born December...,"[appointment-count, born-on]",searchresults#officer,{u'self': u'/officers/QlOUnW4UutpziM_Vf4LNvfeI...,"{u'snippet': [], u'title': [1, 4, 6, 12, 14, 19]}",,Paul Michael BERTIE
0,"{u'country': u'PO31 7RB', u'region': u'Isle Of...","2 Blackberry Lane, Cowes, Isle Of Wight, PO31 7RB",1,"{u'year': 1949, u'month': 12}",Total number of appointments 1 - Born December...,"[appointment-count, born-on]",searchresults#officer,{u'self': u'/officers/9is1nvd-Y4QW6YNGbMfn1GOh...,"{u'snippet': [], u'title': [1, 4, 6, 12, 14, 19]}",,Paul Michael BERTIE
0,"{u'premises': u'The Riverside Centre', u'count...","The Riverside Centre, The Quay, Newport, Isle ...",4,"{u'year': 1969, u'month': 6}",Total number of appointments 4 - Born June 1969,"[appointment-count, born-on]",searchresults#officer,{u'self': u'/officers/Nf6kJZod96K9dIkczLFeLqEA...,"{u'snippet': [], u'title': [1, 4, 6, 11, 13, 18]}",,Paul Andrew FULLER


In [41]:
companies=pd.DataFrame()
localAppointments=appointments[appointments['address_snippet'].str.contains(localarea)]
for appointment in localAppointments['links'].apply(pd.Series)['self'].tolist():
    ddx=pd.DataFrame.from_dict(ch_getAppointments(appointment)['items'])
    tmp=pd.concat([ddx.drop(['appointed_to','name_elements','links','address'], axis=1),
                   ddx['appointed_to'].apply(pd.Series),ddx['address'].apply(pd.Series)], axis=1)
    companies=pd.concat([companies,tmp])

In [42]:
corecols=['name','company_status','company_number','company_name','appointed_on','resigned_on']

localcos=companies[corecols].reset_index(drop=True)
localcos=localAppointments[['title','address_snippet']].merge(localcos, left_on='title',right_on='name')[corecols+['address_snippet']]
localcos.head()

,name,company_status,company_number,company_name,appointed_on,resigned_on,address_snippet
0,Clare Louise MOSDELL,dissolved,06802103,HENLEY HIRE LIMITED,2009-01-27,NaN,"14 Carter Street, Sandown, Isle Of Wight, Unit..."
1,Clare Louise MOSDELL,active,06080566,ST. JOHNS COURT (ISLE OF WIGHT) LIMITED,2008-11-21,2011-05-01,"14 Carter Street, Sandown, Isle Of Wight, Unit..."
2,Clare Louise MOSDELL,active,06080566,ST. JOHNS COURT (ISLE OF WIGHT) LIMITED,2008-11-21,2011-05-01,"14 Carter Street, Sandown, Isle Of Wight, Unit..."
3,Clare Louise MOSDELL,active,09997758,WENTWORTH BOOKKEEPING LTD,2016-02-10,NaN,"14 Carter Street, Sandown, Isle Of Wight, Unit..."
4,Clare Louise MOSDELL,active,09995094,WENTWORTH BOOKING LIMITED,2016-02-09,NaN,"14 Carter Street, Sandown, Isle Of Wight, Unit..."


In [43]:
#Save company data file
localcos.to_csv(companiesfilename,index=False)

In [45]:
len(appointments)

373

In [46]:
fullcompanies=pd.DataFrame()
for appointment in appointments['links'].apply(pd.Series)['self'].tolist():
    ddx=pd.DataFrame.from_dict(ch_getAppointments(appointment)['items'])
    tmp=pd.concat([ddx.drop(['appointed_to','name_elements','links','address'], axis=1),
                   ddx['appointed_to'].apply(pd.Series),ddx['address'].apply(pd.Series)], axis=1)
    fullcompanies=pd.concat([fullcompanies,tmp])

fullcos=companies[corecols].reset_index(drop=True)
fullcos=localAppointments[['title','address_snippet']].merge(fullcos, left_on='title',right_on='name')[corecols+['address_snippet']]
fullcos.to_csv('full'+companiesfilename,index=False)

In [51]:
fullcos[100:120]

,name,company_status,company_number,company_name,appointed_on,resigned_on,address_snippet
100,Malcolm ROSS,active,01274280,STEVE ROSS FOUNDATION FOR THE ARTS,1993-06-07,1993-11-17,"7 Holmes Park Gardens, Kilmarnock, Ayrshire, S..."
101,Malcolm ROSS,active,01274280,STEVE ROSS FOUNDATION FOR THE ARTS,1993-06-07,1993-11-17,"Greenways Forshaw Heath Lane, Forshaw, Earlswo..."
102,Malcolm ROSS,active,01274280,STEVE ROSS FOUNDATION FOR THE ARTS,1993-06-07,1993-11-17,"10 Brittons Close, Sharnbrook, Bedford, Bedfor..."
103,Malcolm ROSS,active,01274280,STEVE ROSS FOUNDATION FOR THE ARTS,1993-06-07,1993-11-17,"New Masonic Hall, Alexandra Rd, Gateshead, Tyn..."
104,Malcolm ROSS,active,01274280,STEVE ROSS FOUNDATION FOR THE ARTS,1993-06-07,1993-11-17,"245-249 Hospital Street, Birmingham, United Ki..."
105,Malcolm ROSS,active,01274280,STEVE ROSS FOUNDATION FOR THE ARTS,1993-06-07,1993-11-17,"19 Hunters Way, Cumnock, Ayrshire, KA18 1AL"
106,Malcolm ROSS,active,01274280,STEVE ROSS FOUNDATION FOR THE ARTS,1993-06-07,1993-11-17,"Ludgate House, Ludgate House, Hornbeam Lane, C..."
107,Malcolm ROSS,active,01274280,STEVE ROSS FOUNDATION FOR THE ARTS,1993-06-07,1993-11-17,"340a Aldridge Road, Streetly, Sutton Coldfield..."
108,Malcolm ROSS,active,01274280,STEVE ROSS FOUNDATION FOR THE ARTS,1993-06-07,1993-11-17,"Rilochan, Rogart, Sutherland, IV28 3YF"
109,Malcolm ROSS,active,01274280,STEVE ROSS FOUNDATION FOR THE ARTS,1993-06-07,1993-11-17,"1 Charlotte Street, Kirkcaldy, KY1 1RN"
